## Classification

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from conf import PROFILE_DIR
from src.img_preprocessing import tf_clean_profile, overview

In [ ]:
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

In [ ]:
# Define CNN architecture
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.Rescaling(1.0 / 255, input_shape=(256,256, 1)),
    keras.layers.Conv2D(64, 7, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(6, activation="softmax")
])

In [ ]:
# Compile model
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
    metrics=["accuracy"]
)

In [ ]:
# Model summary
model.summary()

In [ ]:
# Plot model
# keras.utils.plot_model(model)

In [ ]:
# Load training set
train_ds = keras.preprocessing.image_dataset_from_directory(
  PROFILE_DIR,
  validation_split=0.1,
  subset="training",
  seed=0,
)

In [ ]:
# Load validation set
val_ds = keras.preprocessing.image_dataset_from_directory(
  PROFILE_DIR,
  validation_split=0.1,
  subset="validation",
  seed=0,
)

In [ ]:
# Classes
class_names = train_ds.class_names
print(class_names)

In [ ]:
# Overview before preprocessing
overview(train_ds, class_names)

In [ ]:
# Preprocess images
train_ds = train_ds.map(tf_clean_profile)
train_ds = train_ds.map(lambda x, y: (tf.image.rgb_to_grayscale(x), y))
val_ds = val_ds.map(tf_clean_profile)
val_ds = val_ds.map(lambda x, y: (tf.image.rgb_to_grayscale(x), y))

In [ ]:
# Overview after preprocessing
overview(train_ds, class_names, cmap="gray")

In [ ]:
# Configure dataset for performance
# AUTOTUNE = tf.data.experimental.AUTOTUNE

# train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Train model
early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[early_stopping_cb]
)

In [ ]:
# Plot learning curves
fig, ax = plt.subplots(figsize=(8, 5))
pd.DataFrame(history.history).plot(ax=ax)
ax.grid()
ax.set(ylim=(0, 1))

In [ ]:
# Save model
model.save("models/clf.h5")